In [4]:
import pandas as pd

# Assignment 1: pd.Concat

Combine the 2014 and 2015 data you wrote out in the last section into a single dataframe. 

Then delete the transactions DataFrame (there is a handy base Python keyword for this).

In [7]:
# Code to read in transactions data if you need to re-create CSV/Excel files from section 8 
# NOTE: You won't have the extra columns we created but it won't matter for this assignment

transactions = pd.read_csv("../../retail/transactions.csv", parse_dates=["date"])

#Excel

with pd.ExcelWriter("DataForChandler.xlsx") as writer:
    for year in range(2013, 2018):
        transactions.loc[transactions["date"].dt.year == year].to_excel(
            writer, sheet_name=str(year)
        )
        
# CSV  

# for year in range(2013, 2018):
#     transactions.loc[transactions["date"].dt.year == year].to_csv(
#         f"transactions_{year}.csv"
#     )

In [33]:
transactions2014_2015 = pd.concat([ 
    pd.read_excel("DataForChandler.xlsx", sheet_name="2014"), 
    pd.read_excel("DataForChandler.xlsx", sheet_name="2015") 
], ignore_index=True).drop(["Unnamed: 0"], axis=1)

In [34]:
transactions2014_2015.head()

,date,store_nbr,transactions
0,2014-01-01,25,840
1,2014-01-01,36,487
2,2014-01-02,1,1875
3,2014-01-02,2,2122
4,2014-01-02,3,3350


In [35]:
transactions2014_2015.tail()

,date,store_nbr,transactions
35479,2015-12-31,49,3828
35480,2015-12-31,50,2948
35481,2015-12-31,51,2892
35482,2015-12-31,53,2300
35483,2015-12-31,54,1572


# Assignment 2: Joins

Can you join retail.csv with stores.csv? You'll need to read both files in.

Once you have that, plot:
* Total sales by city, 
* The sum of sales by “type” over time,
* A stacked bar chart with average daily sales by type by month, with “type” as the “stacks”. 
